# Evaluate Hassner's CNN

reference: https://talhassner.github.io/home/projects/cnn_agegender/CVPR2015_CNN_AgeGenderEstimation.pdf

In [1]:
!pip install wandb -qqq

In [2]:
# import necessary libraries
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from wandb.keras import WandbCallback

import pandas as pd
import tensorflow as tf
import wandb

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
defaults = {
    'epochs': 20,
    'batch_size': 64,  # 128 is too large
    'seed': 7,
    'learning_rate': 1e-3,
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'optimizer': 'adam',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

run = wandb.init(
    name='hassner_cnn',
    project='cz4042',
    config=defaults,
)

config = wandb.config

wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)


In [5]:
tf.random.set_seed(config.seed)

In [6]:
# Load dataset as dataframe
base_path = '/content/drive/My Drive/NN Project/'
df = pd.read_csv(base_path + 'aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=config.seed)

# Load images into keras image generator 
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

Found 9755 validated image filenames.
Found 2439 validated image filenames.


In [7]:
def get_hassner_cnn() -> tf.keras.Sequential:
    '''
    Creates Hassner's CNN. Needs to be compiled before using.
    '''
    pool_size = (3, 3)
    strides = (2, 2)

    # custom layer
    input0 = tf.keras.layers.InputLayer(
        input_shape=(224, 224, 3),
    )
    
    conv1 = tf.keras.layers.Conv2D(
        filters=96,
        kernel_size=7,
        activation=config.hidden_activation,
    )

    pool1 = tf.keras.layers.MaxPool2D(
        pool_size=pool_size,
        strides=strides,
    )

    norm1 = tf.keras.layers.Lambda(
        tf.nn.local_response_normalization,
    )

    conv2 = tf.keras.layers.Conv2D(
        filters=256,
        kernel_size=5,
        activation=config.hidden_activation,
    )

    pool2 = tf.keras.layers.MaxPool2D(
        pool_size=pool_size,
        strides=strides,
    )

    norm2 = tf.keras.layers.Lambda(
        tf.nn.local_response_normalization,
    )

    conv3 = tf.keras.layers.Conv2D(
        filters=384,
        kernel_size=3,
        activation=config.hidden_activation,
    )

    pool5 = tf.keras.layers.MaxPool2D(
        pool_size=pool_size,
        strides=strides,
    )
    
    # custom layer
    flatten5 = tf.keras.layers.Flatten()

    fc6 = tf.keras.layers.Dense(
        units=512,
        activation=config.hidden_activation,
    )

    drop6 = tf.keras.layers.Dropout(rate=0.5)

    fc7 = tf.keras.layers.Dense(
        units=512,
        activation=config.hidden_activation,
    )

    drop7 = tf.keras.layers.Dropout(rate=0.5)

    fc8 = tf.keras.layers.Dense(
        units=1,
        activation=config.output_activation,
    )

    return tf.keras.Sequential([
        input0,
        conv1,
        pool1,
        norm1,
        conv2,
        pool2,
        norm2,
        conv3,
        pool5,
        flatten5,
        fc6,
        drop6,
        fc7,
        drop7,
        fc8,
    ])

In [8]:
model = get_hassner_cnn()

model.compile(
    optimizer=config.optimizer,
    loss=config.loss_function,
    metrics=config.metrics,
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 218, 218, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 108, 108, 96)      0         
_________________________________________________________________
lambda (Lambda)              (None, 108, 108, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 256)     614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 51, 51, 256)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 51, 51, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 384)       8

In [9]:
train_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)

Epoch 1/20
153/153 [==============================] - 244s 2s/step - loss: 0.7104 - accuracy: 0.5665 - val_loss: 0.6169 - val_accuracy: 0.6511
Epoch 2/20
153/153 [==============================] - 339s 2s/step - loss: 0.6089 - accuracy: 0.6600 - val_loss: 0.6052 - val_accuracy: 0.6425
Epoch 3/20
153/153 [==============================] - 400s 3s/step - loss: 0.5514 - accuracy: 0.7116 - val_loss: 0.4771 - val_accuracy: 0.7585
Epoch 4/20
153/153 [==============================] - 427s 3s/step - loss: 0.4906 - accuracy: 0.7585 - val_loss: 0.4418 - val_accuracy: 0.7884
Epoch 5/20
153/153 [==============================] - 477s 3s/step - loss: 0.4153 - accuracy: 0.8113 - val_loss: 0.3907 - val_accuracy: 0.8139
Epoch 6/20
153/153 [==============================] - 331s 2s/step - loss: 0.3522 - accuracy: 0.8438 - val_loss: 0.3587 - val_accuracy: 0.8409
Epoch 7/20
153/153 [==============================] - 467s 3s/step - loss: 0.3100 - accuracy: 0.8650 - val_loss: 0.3405 - val_accuracy: 0.8524

In [10]:
results = model.evaluate(
    test_generator,
    callbacks=[WandbCallback()],
)

39/39 [==============================] - 38s 971ms/step - loss: 0.4080 - accuracy: 0.8766


In [11]:
save_path = base_path + '5. Saved Weights and Models/Evaluation/hassner_cnn_eval.h5'
model.save(save_path)

In [12]:
run.finish()

epoch,19
loss,0.05991
accuracy,0.97652
val_loss,0.40798
val_accuracy,0.87659
_step,19
_runtime,6020
_timestamp,1606108611
best_val_loss,0.31979
best_epoch,10


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁
accuracy,▁▃▃▄▅▆▆▆▇▇▇█████████
val_loss,██▅▄▃▂▁▂▁▁▁▁▂▃▄▃▃▄▄▃
val_accuracy,▁▁▄▅▆▇▇▇▇▇██████████
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▃▄▄▅▅▅▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▃▄▄▅▅▅▆▆▆▇▇▇▇███
